**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Optimal taxation with government consumption](#toc1_)    
- 2. [Problem 2: Labor adjustment costs](#toc2_)    
- 3. [Problem 3: Global optimizer with refined multi-start](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

The following packages need to be imported for the code to work.

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import fsolve
from scipy.optimize import minimize

import warnings 
warnings.filterwarnings("ignore", category=RuntimeWarning)

## 1. <a id='toc1_'></a>[Problem 1: Optimal taxation with government consumption](#toc0_)


Consider a worker choosing hours of labor, $L\in[0,24]$, to maximize utility: 

$$
\begin{align*}
V(w,\tau,G)&=\max_{L\in[0,24]}\ln\left(C^{\alpha}G^{1-\alpha}\right)-\nu\frac{L^{2}}{2}\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}
$$

where 

* $C$ is *private* consumption with weight $\alpha\in(0,1)$.
* $\kappa > 0$ is the *free private* consumption component.
* $(1-\tau)wL$ is the *costly private* consumption component.
* $w > 0 $ is the real wage.
* $\tau \in (0,1)$ is the labor-income tax rate.
* $G > 0 $ is *government* consumption with weight $1-\alpha$.
* $\nu > 0$ is the disutility of labor scaling factor


The baseline parameters are:

$$
\begin{align*}
\alpha &= 0.5\\
\kappa &= 1.0\\
\nu &= \frac{1}{2\cdot16^2} \\
w &= 1.0 \\ 
\tau &= 0.30 \\
\end{align*}
$$

**Question 1:** Verify that the optimal labor supply choice is $L^{\star}(\tilde{w}) =\frac{-\kappa+\sqrt{\kappa^{2}+4\frac{\alpha}{\nu}\tilde{w}^2}}{2\tilde{w}}$, where $\tilde{w} = (1-\tau)w$, for $G\in\left\{1.0 , 2.0\right\}$.

To answer this question we start by setting up the model as describes above. First we set the baseline parameters: alpha, kappa, nu, w, and tau to their baseline values. The values of G are set to [1.0, 2.0]. We define the optimal labor supply choice given the values of alpha, kappa, nu, and $\tilde{w}$.

We create a loop to calculate optimal labor supply for each G.

In [ ]:
# Define the parameters
alpha = 0.5
kappa = 1.0
nu = 1 / (2 * 16**2)
w = 1.0
tau = 0.3
G_values = [1.0, 2.0]

# Define the optimal labor supply choice
def L_star(tilde_w):
    return (-kappa + np.sqrt(kappa**2 + 4 * alpha / nu * tilde_w**2)) / (2 * tilde_w)

for G in G_values:
    tilde_w = (1 - tau) * w

    print(f"For G = {G}:")
    print(f"Optimal Labor Hours: {L_star(tilde_w):.3f}")

We calculate the optimal labor supply and find that the optimal labor hours amount to $15.30$ regardless of whether G equals 1 or 2.

There is no difference in the optimal labor hours for varying G values. Changes in government consumption do not influence the worker's decision about their labor supply, according to the assumptions underpinning this model.

**Question 2:** Illustrate how $L^{\star}(\tilde{w})$ depends on $w$.

We start by defining the array of 1,000 equally spaced real wage values ranging from 0.01 to 5.0. We redefine tilde_w to calculate the real wage for the range of w. 

To illustrate the optimal labor supply as a function of real wage we uses Plotly's 'go.Figure' class. Go.Layout is used decide the layout of the plot, including the title and axis titles. It also formats the axes to display only two decimal places.

In [ ]:
# Set values of w and tilde w
w_values = np.linspace(0.01, 5.0, 1000)
tilde_w_values = (1 - tau) * w_values

# Create interactive plots using go.Figure
fig = go.Figure(data=go.Scatter(x=w_values, y=L_star(tilde_w_values), mode='lines'))
fig.update_layout(title='Labor supply', 
    xaxis_title="Wage (w)",
    yaxis_title="Labor Hours (L)",
    xaxis=dict(tickformat=".2f"),
    yaxis=dict(tickformat=".2f"))
fig.show()

The plot illustrates the optimal labor supply as a function of the real wage. It shows that as the wage increases, the number of labor hours also increases, thus confirming the positive relationship between wage and labor supply. However, it's important to note the concave nature of the graph. This implies that the marginal effect of an additional unit of wage on labor supply diminishes as the wage increases. Ultimately, the number of labor hours tends to converge towards 16 as the wage continues to rise.


We now consider a government, who chooses $\tau$ and spend all of the taxes on government consumption so:

$$
G = \tau w L^{\star}((1-\tau)w)
$$

**Question 3:** Plot the implied $L$, $G$ and worker utility for a grid of $\tau$-values.


To illustrate how L, G, and worker utility depend on the tax rate, we start by the worker's utility as a functino of labor, tau and G. the constrain, C, is also defined. We define the range of the tax rate between 0 and 1 and the government consumption using the given equation. Labor hours and utility is calculated with the new values of L, tau and G. 

Finally, we create three plots - one each for optimal labor, government consumption, and worker utility. We use the 'go.Figure' function, as in the previous question, to generate these plots. This enables us to see how these key economic variables change as the tax rate varies.

In [ ]:
# Define the worker utility
def utility(L, tau, G):
    C = kappa + (1 - tau) * w * L
    return np.log(C**alpha * G**(1 - alpha)) - nu * L**2 / 2

# Set tau values and define government consumption
tau_values = np.linspace(0.001, 1, 1000)  # range of tau values
L_values = L_star((1 - tau_values) * w)
G_values = tau_values * w * L_values
utility_values = utility(L_values, tau_values, G_values)

# Create interactive plots using go.Figure
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=("Labor Supply", "Government Consumption", "Worker Utility"),
                    horizontal_spacing=0.1)  


# Adding traces
fig.add_trace(go.Scatter(x=tau_values*100, y=L_values, mode='lines'), row=1, col=1)
fig.add_trace(go.Scatter(x=tau_values*100, y=G_values, mode='lines'), row=1, col=2)
fig.add_trace(go.Scatter(x=tau_values*100, y=utility_values, mode='lines'), row=1, col=3)

# Updating xaxis and yaxis properties for each subplot
fig.update_xaxes(title_text="Tax Rate, %", row=1, col=1, tickformat=".2f")
fig.update_yaxes(title_text="Labor Hours", row=1, col=1, tickformat=".2f")

fig.update_xaxes(title_text="Tax Rate, %", row=1, col=2, tickformat=".2f")
fig.update_yaxes(title_text="Government Consumption", row=1, col=2, tickformat=".2f")

fig.update_xaxes(title_text="Tax Rate, %", row=1, col=3, tickformat=".2f")
fig.update_yaxes(title_text="Utility", row=1, col=3, tickformat=".2f")

# Updating layout size and title
fig.update_layout(title_text="Impact of Tax Rate", showlegend=False)

fig.show()

For the plot of optimal labor hours, we observe how labor hours depend negatively on the tax rate. There is a negative relationship between the two variables, demonstrating that a higher tax rate corresponds to fewer labor hours. This is the converse of the relationship shown in Question 2, but the underlying principle is the same. As taxes increase, the real wage decreases, reducing the incentive to work longer hours.

For the plot of government consumption illustrates how as he tax rate increases, we see an increase in government consumption, up until the tax rate reaches approximately 84 percent. This trend likely reflects the increased revenue the government accrues from higher taxation. However, beyond this point, government consumption decreases rapidly. This decline in government consumption could be due to decreased labor hours due to high taxes, which effectively shrink the tax base and consequently lower government revenue and consumption.

The plot demonstrating the worker's utility as a function of the tax rate takes the shape of an inverted U-function. This indicates that there is an optimal tax rate at which the worker's utility is maximized. From the plot, the maximum utility is achieved when the tax rate is approximately 50 percent.

**Question 4:** Find the socially optimal tax rate $\tau^{\star}\in(0,1)$ maximizing worker utility. Illustrate your result.

To find the optimal tax rate that maximizes worker utility, we start by defining the objective function that needs to be optimized. In this case, the objective function is defined as the negative of the utility function because we are performing a minimization to maximize utility. 

We use the minimize function to perform the actual optimization. This function searches for the value of tau that minimizes the negative objective function. It uses a bounded method, which means it only searches between 0 and 1 as the tax rate are defined to be between 0 and 1. The results of the optimization are stored in optimal_tau, which contains information about the outcome.

In [ ]:
objective = lambda tau: -utility(L_star((1 - tau) * w), tau, tau * w * L_star((1 - tau) * w))
result = minimize(objective, 0.5, bounds=[(0, 1)])
optimal_tau = result.x[0]*100

print(f"Optimal tau is {optimal_tau:.3f} pct.")

The worker´s utility is maximized when the tax rate equals 51.45 pct. This means that 51.45 pct of the worker´s wage is allocated to government consumption. 

To illustrate the optimal tax rate, we use go.Figure as in question 3. 

In [ ]:
# Create the figure
fig = go.Figure()

# Add traces for each kappa
fig.add_trace(go.Scatter(x=tau_values*100, y=utility_values, mode='lines', name='Utility'))

# Add a marker for the optimal tau
fig.add_trace(go.Scatter(x=[optimal_tau], y=[-objective(optimal_tau/100)], mode='markers', name='Optimal Tax Rate'))

# Set labels
fig.update_layout(title='Utility as a Function of the Tax Rate', 
                  xaxis_title='Tax Rate, %', 
                  yaxis_title='Utility',
                  xaxis=dict(tickformat=".2f"), 
                  yaxis=dict(tickformat=".2f"))

# Show the figure
fig.show()

A more general preference formulation for the worker is:

$$
\begin{align*}
\mathcal{V}(w,\tau,G)&=\max_{L\in[0,24]}\frac{\left[ \left( \alpha C^{\frac{\sigma-1}{\sigma}}+(1-\alpha) G^{\frac{\sigma-1}{\sigma}} \right)^{\frac{\sigma}{\sigma-1} }\right]^{1-\rho}-1}{1-\rho}- \nu\frac{L^{1+\varepsilon}}{1+\varepsilon},\,\,\,\varepsilon,\rho,\sigma>0,\,\,\,\rho,\sigma\neq1\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}    
$$

Optimal labor supply is now $L^{\star}(\tilde{w},G)$.

Questions 5 and 6 must be answered with the general formulation, and for 2 different set of parameters:

- Set 1:  $\sigma = 1.001$, $\rho = 1.001$ and $\varepsilon = 1.0$.
- Set 2:  $\sigma = 1.5$, $\rho = 1.5$ and $\varepsilon = 1.0 $.

**Question 5:** Find the $G$ that solves $G = \tau w L^{\star}((1-\tau)w,G)$ using the $\tau$ found in question 4.

*Hint: First write code that solves the worker problem for given values of $G$ and $\tau$. Then find the correct G based on this.*

We start by defining the new parameters, sigma, rho and epsilon with two values each. Then the worker's general preference formulation is defined as a function of labor, wage, government consumption, sigma, rho and epsilon. d_utility_general defines the derivative of the utility with respect to labor L. We use this function in L_star_general to find the optimal labour that maxmizes worker utility.

We loop over the two sets of parameters and find the G that solves the equation. 

In [ ]:
# Define values for sigma, rho, and epsilon
sigma_values = [1.001, 1.5]
rho_values = [1.001, 1.5]
epsilon_values = [1.0, 1.0]

# Define utility function for general case
def utility_general(L, tilde_w, G, sigma, rho, epsilon):
    C = kappa + tilde_w * L
    term = (alpha * C**((sigma - 1) / sigma) + (1 - alpha) * G**((sigma - 1) / sigma))**(sigma / (sigma - 1))
    return (term**(1 - rho) - 1) / (1 - rho) - nu * L**(1 + epsilon) / (1 + epsilon)

# Define the derivative of the utility function
def d_utility_general(L, tilde_w, G, sigma, rho, epsilon):
    C = kappa + tilde_w * L
    term = alpha * C**((sigma - 1) / sigma) + (1 - alpha) * G**((sigma - 1) / sigma)
    return (alpha * tilde_w * term**(1 / (sigma - 1)) * C**(-1 / sigma) - nu * epsilon * L**epsilon) * term**(1 - rho)

# Function to compute optimal labor that maximizes the utility function
def L_star_general(tilde_w, G, sigma, rho, epsilon):
    result = minimize(lambda L: -utility_general(L, tilde_w, G, sigma, rho, epsilon), 0.1, jac=lambda L: -d_utility_general(L, tilde_w, G, sigma, rho, epsilon), bounds=[(0, 24)])
    return result.x[0]

# Function to compute the level of government consumption that solves the budget constraint
def G_star(sigma, rho, epsilon):
    # Initial guess 
    G_star_initial_guess = 1
    return fsolve(lambda G: G - optimal_tau * w * L_star_general((1 - optimal_tau) * w, G, sigma, rho, epsilon), G_star_initial_guess)[0]

# Loop through the two sets of parameters values
for sigma, rho, epsilon in zip(sigma_values, rho_values, epsilon_values):
    G_star_value = G_star(sigma, rho, epsilon)
    print(f"For sigma={sigma}, rho={rho}, epsilon={epsilon}, the level of government consumption G* that solves the equation is: {G_star_value:.3f}")


For both sets of parameters, we find that the government consumption equals 5.15. This result could imply that government consumption does not depend on sigma, rho, and epsilon. Furthermore, since we are retaining the optimal value of tau, it might also indicate that government consumption depends solely on wages and tau.

Additionally, it's important to note that the initial guess significantly influences the level of government consumption. This could suggest that the function has multiple local optima.

**Question 6:** Find the socially optimal tax rate, $\tau^{\star}$, maximizing worker utility, while keeping $G = \tau w L^{\star}((1-\tau)w,G)$.

In question 5 we used the optimal tau and we should now fin the optimal tax rate. We redefine government consumption by adding tau to the function. We do the same for utility. The last part of the code loops over the two sets of parametera and print the found values of tau. 

In [ ]:
# Function to compute the level of government consumption that satisfies the budget constraint for a given tau
def G_star(tau, sigma, rho, epsilon):
    G_star_initial_guess = 1
    return fsolve(lambda G: G - tau * w * L_star_general((1 - tau) * w, G, sigma, rho, epsilon), G_star_initial_guess)[0]

# Function to compute the utility as a function of tau
def utility_tau(tau, sigma, rho, epsilon):
    G = G_star(tau, sigma, rho, epsilon)
    tilde_w = (1 - tau) * w
    return -utility_general(L_star_general(tilde_w, G, sigma, rho, epsilon), tilde_w, G, sigma, rho, epsilon)

# Loop through the two sets of parameters values
for sigma, rho, epsilon in zip(sigma_values, rho_values, epsilon_values):
    # Minimize the utility function with respect to tau
    result = minimize(utility_tau, 0.1, args=(sigma, rho, epsilon), bounds=[(0, 1)])
    optimal_tau_general = result.x[0]
    print(f"For sigma={sigma}, rho={rho}, epsilon={epsilon}, the optimal tax rate is: {optimal_tau_general*100:.3f}")

We see that the optimal tax rate equals 58,43 pct. for sigma = 1.001, rho = 1.001 and epsilon = 1. 

The optimal tax rate is lower, 56.58 pct. when sigma = 1.5, rho = 1.5 and epsilon = 1. It means that the parameters has a significant impact on the tax rate. 

## 2. <a id='toc2_'></a>[Problem 2: Labor adjustment costs](#toc0_)

You own a hair salon. You employ hairdressers, $\ell_t$, to produce haircuts, $y_t = \ell_t$.

The wage for each haridresser is $w$.

The demand for haircuts implies that the price of haircuts you can charge is $p_t = \kappa_t y_t^{-\eta}$, where $\kappa_t$ is a demand-shock and $\eta \in (0,1)$ measures the elasticity of demand.

Profits are:

$$
\Pi_t = p_t y_t - w \ell_t = \kappa_t \ell_t^{1-\eta} - w \ell_t
$$

Baseline parameters are:
- $\eta = 0.5$
- $w = 1.0$

**Question 1:** Verify numerically that $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ maximises profits, for $\kappa\in\left\{1.0 , 2.0\right\}$.

To verify this numerically, we start by creating a profit function that depends on the variables: l, kappa, eta, and the wage. We also define the optimal number of hairdressers, denoted as 'l.' Additionally, we set the parameter values for eta and the wage to 0.5 and 1.0, respectively. Furthermore, we assign the values 1.0 and 2.0 to the kappa variable.

Next, we create a loop to iterate over different kappa values in order to find the optimal number of hairdressers and the maximum profit. The resulting values are printed below.

In [ ]:
# define parameters
eta = 0.5
w = 1.0
kappa_values = [1.0, 2.0]

# define profit function
def profit(l, kappa, eta, w):
    return kappa*l**(1-eta) - w*l

# define function to calculate optimal l
def optimal_l(kappa, eta, w):
    return ((1-eta)*kappa/w)**(1/eta)

# calculate optimal l and maximum profit for each kappa
for kappa in kappa_values:
    l_opt = optimal_l(kappa, eta, w)
    max_profit = profit(l_opt, kappa, eta, w)
    print(f"For kappa={kappa}, optimal l={l_opt} with maximum profit={max_profit}")

For kappa = 1, the optimal number of employed hairdressers is 0.25, and for kappa = 2, the optimal number of hairdressers is 1. It makes sense if kappa describes the demand, as the higher the demand, the greater the need for hairdressers.

By running the code below, the profit as a function of the number of hairdressers is illustrated.
First, we define the range for the number of hairdressers and calculate the profit values for the two kappa values. From these values, we find the optimal number of 'l' and the maximum profit. We use the 'zip' function to iterate over multiple lists, similar to question 2. We use 'go.Figure' to plot.

In [ ]:
# Define the range of l (number of hairdressers)
l_values = np.linspace(0, 2, 400)

# Calculate profit values for different kappa
profit_values_kappa1 = profit(l_values, kappa_values[0], eta, w)
profit_values_kappa2 = profit(l_values, kappa_values[1], eta, w)

# Calculate optimal l and max profit for each kappa
optimal_l_values = [optimal_l(kappa, eta, w) for kappa in kappa_values]
max_profit_values = [profit(l, kappa, eta, w) for l, kappa in zip(optimal_l_values, kappa_values)]

# Create the figure
fig = go.Figure()

# Add traces for each kappa
fig.add_trace(go.Scatter(x=l_values, y=profit_values_kappa1, mode='lines', name=f'kappa = {kappa_values[0]}'))
fig.add_trace(go.Scatter(x=l_values, y=profit_values_kappa2, mode='lines', name=f'kappa = {kappa_values[1]}'))

# Add markers for the optimal l
fig.add_trace(go.Scatter(x=optimal_l_values, y=max_profit_values, mode='markers', name='Optimal l'))

# Set labels
fig.update_layout(title='Profit for different numbers of hairdressers', 
                  xaxis_title='Number of hairdressers, l', 
                  yaxis_title='Profit',
                  xaxis=dict(tickformat=".2f"), 
                  yaxis=dict(tickformat=".2f"))

# Show the figure
fig.show()

The plot shows the profit function for different demand shocks. For kappa equals 1, we can observe that the optimal number of hairdressers is 0.25, and the hairsalon will incur either zero profit or negative profit when employing 1 hairdresser or more.

For kappa equals 2, the optimal number of hairdressers is 1, resulting in a profit of 1. Within this range of l, there will not be any negative profit.

We now consider a *dynamic* version of the model.

* The demand-shock is a so-called AR(1) in logs, 

$$
\log \kappa_{t} = \rho \log \kappa_{t-1} + \epsilon_{t},\,\,\, \epsilon_{t+1} \sim \mathcal{N}(-0.5\sigma_{\epsilon}^2,\sigma_{\epsilon})
$$

* Any hiring or firing implies a fixed adjustment cost, $\iota > 0 $.
* Future profits are discounted with a monthly factor of $R \in (0,1)$.

The initial demand shock is $\kappa_{-1} = 1$ and the planning horizon is 10 years, i.e. 120 months so $t \in \{0,1,2,\dots,119\}$. Initially you don't have any employees, $\ell_{-1}=0$


The *ex post* value of the salon is *conditional* on the shock series is:

$$
h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119}) = \left[\sum_{t=0}^{119}R^{-t}\left[\kappa_{t}\ell_{t}^{1-\eta}-w\ell_{t}-\boldsymbol{1}_{\ell_{t}\neq\ell_{t-1}}\iota\right]\right]
$$

The *ex ante* expected value of the salon can be approximated by

$$
H = \mathbb{E}[h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119})] \approx \frac{1}{K}\sum_{k=0}^{K} h(\epsilon_0^k,\epsilon_1^k,\dots,\epsilon_{119}^k)
$$

where each $k\in\{0,1,\dots,K-1\}$ is a random shock series. Maximizing profitability means maximizing $H$.


Baseline parameters are: 

- $\rho = 0.90$
- $\iota = 0.01$
- $\sigma_{\epsilon} = 0.10$
- $R = \left(1+0.01\right)^{1/12}$

**Question 2:** Calculate $H$ if the policy  $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ from question 1 is followed. Choose $K$ so the approximation is good enough to not affect your results substantially.

We will now consider a dynamic version of the model, looking over a period of 120 months. Changes in employment come with an adjustment cost, denoted by $i$. This cost is defined by an indicator function. The function equals 1 if there is a change in employment from period $t-1$ to period $t$, and equals 0 if there is no change.

We start by defining the baseline parameters, T and K. We set K equal to 10,000 to allow for a large number of simulations that accurately capture the distribution of outcomes. We create arrays for $l_t$ and kappa, initializing them with zeros. These arrays are then used in a loop. For each time period and for each demand shock, we calculate the values for epsilon, kappa, and $l_t$. Epsilon is drawn from a normal distribution as described in the model. After completing the loop, we calculate the profit at each time period for each demand shock. We employ the expression '[:, t]' to index arrays where the operator ':' selects all elements along one dimension and 't' selects the t-th column. Finally, we calculate H, the ex-ante expected value, which is the mean of the profits across all simulations.

In [ ]:
# set parameters
rho = 0.9
iota = 0.01
sigma = 0.1
R = (1+0.01)**(1/12)
T = 120
K = 10000  # example, you may need more for a good approximation

# calculate l_t for each t and k
lt = np.zeros((K, T))
kappa_t = np.zeros((K, T))
kappa_t[:, 0] = 1  # initial kappa
for t in range(1, T):
    epsilon = np.random.normal(loc=-0.5*sigma**2, scale=sigma, size=K)
    kappa_t[:, t] = np.exp(rho * np.log(kappa_t[:, t-1]) + epsilon)
    lt[:, t] = ((1-eta)*kappa_t[:, t]/w)**(1/eta)

# calculate profits
profits = np.zeros((K, T))
for t in range(T):
    profits[:, t] = R**(-t) * (kappa_t[:, t]*lt[:, t]**(1-eta) - w*lt[:, t] - iota*(lt[:, t] != lt[:, t-1]))

# calculate H
H = np.mean(profits)

print(f"Expected value of the salon, H: {H:.3f}")

We find that the expected value of H is approximately 0.23. Since the equations will be run 10,000 times due to the simulation, the exact value might vary slightly. We argue that the result does not change substantially, as any variation is likely to appear in the third decimal place.

When using a lower value for K, the simulation is run fewer times, but we observe a larger difference in the expected value

Next, we consider policies on the form:

$$

\ell_{t}=\begin{cases}
\ell_t^{\ast}  & \text{if }\left|\ell_{t-1}-\ell_t^{\ast} \right|>\Delta\\
\ell_{t-1} & \text{else }
\end{cases}
\\
\text{where}\,\,\ell_t^{\ast} = \left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}} \\

$$
With $\Delta \geq 0$ and $\Delta = 0$ being the previous policy.



**Question 3:** Calculate $H$ if the policy above was followed with $\Delta = 0.05$. Does it improve profitability?

In question two, hairdressers would be hired or fired even for the smallest difference between the optimal number of hairdressers and the current number. This implies that the number of hairdressers would likely change every period. For this question, there needs to be a significant difference between the optimal number of hairdressers and the current number before any changes happen. By implementing $\Delta$, the salon minimizes the cost of hiring and firing people if the demand has only changed insignificantly.

In the following code, we start by defining $\Delta$. As we did in Question 2, we create arrays for lt_new and profit_new that initially contain only zeros. These arrays are then used in two loops: one to calculate the number of hairdressers, looping over each time period and for each demand shock, and another to calculate the profit over each time period. We define a new $H$ as the mean of the new profit.

In [ ]:
# Set new policy parameters
delta = 0.05

# Calculate l_t for each t and k
lt_new = np.zeros((K, T))
for t in range(1, T):
    l_star = ((1-eta)*kappa_t[:, t]/w)**(1/eta)
    lt_new[:, t] = np.where(np.abs(lt_new[:, t-1] - l_star) > delta, l_star, lt_new[:, t-1])

# Calculate profits
profits_new = np.zeros((K, T))
for t in range(T):
    profits_new[:, t] = R**(-t) * (kappa_t[:, t]*lt_new[:, t]**(1-eta) - w*lt_new[:, t] - iota*(lt_new[:, t] != lt_new[:, t-1]))

# Calculate new H
H_new = np.mean(profits_new)

print(f"New expected value of the salon, H: {H_new:.3f}")

We find that the new expected value equals 0.23, which is higher than the expected value we found in question 2. The reason might be that the salon has minimized their costs of hiring and firing employees.

**Question 4:** Find the optimal $\Delta$ maximizing $H$. Illustrate your result.

In question 3, we set $\Delta$ to 0.05, and in this step we aim to find the optimal $\Delta$. We start by creating a range of delta values from 0 to 1. We have tested values higher than 1, but have found that the optimal delta value does not exceed 1. To balance precision and computational efficiency, we generate 1,000 intervals within this range.

Next, we calculate the number of hairdressers, the profit, and the updated H value in a loop for each delta value. We determine the optimal delta value using the np.argmax function, which finds the maximum value of H and the corresponding delta value is stored in the variable optimal_delta.

To see how H depends on the delta values, we plot it by using go.Figure. 

In [ ]:
# define range of delta values
delta_values = np.linspace(0, 1, 1000)

# calculate H for each delta
H_values = []
for delta in delta_values:
    # calculate l_t for each t and k
    lt_new = np.zeros((K, T))
    for t in range(1, T):
        l_star = ((1-eta)*kappa_t[:, t]/w)**(1/eta)
        lt_new[:, t] = np.where(np.abs(lt_new[:, t-1] - l_star) > delta, l_star, lt_new[:, t-1])

    # calculate profits
    profits_new = np.zeros((K, T))
    for t in range(T):
        profits_new[:, t] = R**(-t) * (kappa_t[:, t]*lt_new[:, t]**(1-eta) - w*lt_new[:, t] - iota*(lt_new[:, t] != lt_new[:, t-1]))

    # calculate new H
    H_new = np.mean(profits_new)
    H_values.append(H_new)

# find optimal delta
optimal_delta = delta_values[np.argmax(H_values)]

print(f"Optimal delta: {optimal_delta:.3f}")

In [ ]:
# Create the figure
fig = go.Figure()

# Add trace for H values
fig.add_trace(go.Scatter(x=delta_values, y=H_values, mode='lines', name='H values'))

# Add marker for the optimal delta
fig.add_trace(go.Scatter(x=[optimal_delta], y=[max(H_values)], mode='markers', name='Optimal delta'))

# Set labels
fig.update_layout(title='H values for different delta values', 
                  xaxis_title='Delta values', 
                  yaxis_title='H values', 
                  xaxis=dict(tickformat=".5f"), 
                  yaxis=dict(tickformat=".5f"))

# Show the figure
fig.show()

We find that the optimal value of delta to maximize H is 0.076, which is quite close to the baseline value given in question 3. For delta equal to 0.076, the expected value of the salon is 0.234. We see from the plot that the H values for different delta values around the optimal point are very close to each other. That is why we have used 5 decimal places to actually see the differences. For delta values larger than 0.200, the expected value of the salon converges rather quickly toward 0 as delta approaches 1. This is because the allowable difference between the optimal number of hairdressers and the current number becomes too large, resulting in the salon having too many hairdressers during periods of low demand and too few during periods of high demand.


**Question 5:** Suggest an alternative policy you believe might improve profitability. Implement and test your policy.



As an alternative policy, we suggest adjusting the number of hairdressers based on the average demand over a period of time, rather than solely on a single period's demand. This approach could potentially reduce the frequency of hiring or firing, thereby decreasing the associated costs. We chose a period of three months, which represents a quarter, in hopes of minimizing seasonal variations. 

The implementation of this policy is similar to the code in the previous questions. We start by creating an array for l that initially contains only zeros, and we define the length of the averaging period. We then loop over the time, T, to calculate the average demand over a three-month period and determine the optimal l. Next, we create an array for the profit and use it within the loop over time and the shock series. Finally, the expected value of the salon, defined as the mean of the profits, is calculated.

In [ ]:
# Calculate l_t for each t and k, where l_t is based on the average kappa_t over a 3 month period
lt_new_alt = np.zeros((K, T))
average_period = 3  # 5 months

for t in range(1, T):
    # Calculate average kappa over 3 month period (or less if t < 3)
    kappa_avg = np.mean(kappa_t[:, max(0, t-average_period):t+1], axis=1)
    
    l_star = ((1-eta)*kappa_avg/w)**(1/eta)
    lt_new_alt[:, t] = np.where(np.abs(lt_new_alt[:, t-1] - l_star) > optimal_delta, l_star, lt_new_alt[:, t-1])

# Calculate profits
profits_new_alt = np.zeros((K, T))
for t in range(T):
    profits_new_alt[:, t] = R**(-t) * (kappa_t[:, t]*lt_new_alt[:, t]**(1-eta) - w*lt_new_alt[:, t] - iota*(lt_new_alt[:, t] != lt_new_alt[:, t-1]))

# Calculate new H
H_new_alt = np.mean(profits_new_alt)

print(f"New H with alternative policy: {H_new_alt:.3f}")

We find that the new expected value of the salon with the alternative policy equals 0.23 which is lower than the expected value we foudn in question 4. 

We chose the period of the average demand to be 3 months. Below we plot how the expected value of the salon depends on the lenght of the average period. We set the average period to be from 1 to 12 months as the salon can adapt hte number of employees to longer-term trends in demand. We use nested loops, where we loop over the average priod values and the whole time period. In the loop we calculate l, profit and the expected value of the salon. The plot is made using go.Figure.

In [ ]:
# define range of average_period values
average_period_values = np.arange(1, 13)

# calculate H for each average_period
H_values_alt = []
for average_period in average_period_values:
    # calculate l_t for each t and k, where l_t is based on the average kappa_t over the average_period
    lt_new_alt = np.zeros((K, T))
    for t in range(1, T):
        kappa_avg = np.mean(kappa_t[:, max(0, t-average_period):t+1], axis=1)
        l_star = ((1-eta)*kappa_avg/w)**(1/eta)
        lt_new_alt[:, t] = np.where(np.abs(lt_new_alt[:, t-1] - l_star) > optimal_delta, l_star, lt_new_alt[:, t-1])

    # calculate profits
    profits_new_alt = np.zeros((K, T))
    for t in range(T):
        profits_new_alt[:, t] = R**(-t) * (kappa_t[:, t]*lt_new_alt[:, t]**(1-eta) - w*lt_new_alt[:, t] - iota*(lt_new_alt[:, t] != lt_new_alt[:, t-1]))

    # calculate new H
    H_new_alt = np.mean(profits_new_alt)
    H_values_alt.append(H_new_alt)

# Create the figure
fig = go.Figure()

# Add trace for H values
fig.add_trace(go.Scatter(x=average_period_values, y=H_values_alt, mode='lines', name='H values'))

# Set labels
fig.update_layout(title='H values for different average periods', xaxis_title='Average period (months)', yaxis_title='H values')

# Show the figure
fig.show()

The plot shows that the longer the length of the average period, the smaller the value of the expected value of the salon. When only considering the last period, H equals 0.234. However, when considering the last 12 periods for adjustment, the expected value of the salon decreases to 0.229. This difference might be due to the very small adjustment cost, which makes it relatively inexpensive to adjust the workforce according to the demand.

## 3. <a id='toc3_'></a>[Problem 3: Global optimizer with refined multi-start](#toc0_)

We consider the Griewank function:

$$ f(\boldsymbol{x}) = \sum^n_{i=1} \frac{x^2_i}{4000}-\prod^n_{i=1}\cos\left(\frac{x_i}{\sqrt{i}}\right)+1$$

The **global minimum** of this function is $f(0,0) = 0$ (remember: $\cos(0)=1$).<br>
But the function also have a lot of **local minima**.

In [ ]:
def griewank(x):
    return griewank_(x[0],x[1])
    
def griewank_(x1,x2):
    A = x1**2/4000 + x2**2/4000
    B = np.cos(x1/np.sqrt(1))*np.cos(x2/np.sqrt(2))
    return A-B+1

A **refined global optimizer with multi-start** is:

1. Choose *bounds* for $\mathbf{x}$ and *tolerance* $\tau > 0$.
2. Choose number of *warm-up iterations*, $\underline{K} > 0$ and *maximum number of iterations*, $K > \underline{K}$.
3. In each iteration for $k \in \{0,1,\dots,K-1\}$:

    A. Draw random $\mathbf{x}^k$ uniformly within chosen bounds.

    B. If $k < \underline{K}$ go to step E.

    C. Calculate $\chi^k = 0.50\cdot\frac{2}{1+\exp((k-\underline{K})/100)}$  

    D. Set $\mathbf{x}^{k0} = \chi^k \mathbf{x}^k + (1-\chi^k)\mathbf{x}^{\ast} $

    E. Run optimizer with $\mathbf{x}^{k0}$ as initial guess and $\mathbf{x}^{k\ast}$ as result.

    F. Set $\mathbf{x}^{\ast} = \mathbf{x}^{k\ast}$ if $k = 0$ or $f(\mathbf{x}^{k\ast}) < f(\mathbf{x}^{\ast})$

    G. If $f(\mathbf{x}^{\ast}) < \tau$ go to step 4.

4. Return the result $\mathbf{x}^{\ast}$.

As settings we choose:

* $x_1,x_2 \in  [-600,600]$
* $\tau = 10^{-8}$
* $\underline{K}=10$
* $K=1000$

The optimizer in Step 3.E is `BFGS` with a tolerance of $\tau$.

**Question 1:** Implement the refined global optimizer with multi-start. Illustrate how the effective initial guesses $\mathbf{x}^{k0}$ vary with the iteration counter $k$.

In the following code we try to implement a multi-start optimization approach. The first definition of griewank_ was given in the exam. It defines the Griewank function in two parts, A and B. Second, we set up the parameters where bounds is range of values that the function's parameters can take. Tol is the tolerance for the optimization algorithm which stops if the changes between two iterations is very close to zero. K and K_underline are parameters representing the total number of iterations and the number of warm-up iterations and , respectively.

Then we define the optimizer function which performs the multi-start optimization. It takes the function to the parameters and bounds as arguments. It initializes the best solution (x_star) and the best function value (f_star) so far to none. Then a loop is created that runds K times where for each iteration it generates a initial guess x_k for the parameters, uniformly distributed within the bounds. If the iteration number is greater than or equal to the number of warm-up iterations, it calculates a weighting factor chi_k and generates a new initial guess x_k0 as a weighted combination of the random guess and the best solution so far. The effective intial guesses will be appended to x_k0_list. The BFGS optimization algorithm will run with the function of the initial guess. 
If this is the first iteration or the function value at the result is lower than the best function value so far, it updates x_star and f_star with the result. When the best function value is lower than the tolerance, it breaks the loop.

Finally, it returns the best solution found and the list of all effective initial guesses.

In [ ]:
# Griewank function definition
def griewank(x):
    A = np.sum(x**2/4000)
    B = np.prod(np.cos(x/np.sqrt(np.arange(1, len(x)+1))))
    return A-B+1

# Setup parameters
bounds = np.array([[-600, 600]]*2)
tol = 1e-8
K_underline = 10
K = 1000

# Optimizer function
def optimizer(fun, bounds, tol, K_underline, K):
    x_star = None
    f_star = None
    x_k0_list = []

    for k in range(K):
        # Draw random x^k within chosen bounds
        x_k = np.random.uniform(bounds[:, 0], bounds[:, 1])

        # Calculate chi^k after warm-up iterations
        if k >= K_underline:
            chi_k = 0.5 * (2 / (1 + np.exp((k-K_underline) / 100)))
            x_k0 = chi_k * x_k + (1-chi_k) * x_star
        else:
            x_k0 = x_k

        x_k0_list.append(x_k0)

        # Run BFGS optimizer with x_k0 as initial guess
        result = minimize(fun, x_k0, method='BFGS', tol=tol)

        # Update x_star and f_star if this is the first iteration or if a lower function value is found
        if x_star is None or fun(result.x) < f_star:
            x_star = result.x
            f_star = fun(result.x)

        # Check if f_star is lower than the tolerance
        if f_star < tol:
            break

    return x_star, np.array(x_k0_list)

# Run the optimizer
x_star, x_k0_list = optimizer(griewank, bounds, tol, K_underline, K)

To illustrate how the effective initial guesses $\mathbf{x}^{k0}$ vary with the iteration counter $k$, we plot how the effective initial guesses (x1_k0 and x2_k0) vary with the iteration counter. We uses plotly to create line plot of x_k0_list where y=x_k0_list[:, 0] and y=x_k0_list[:, 1] set the y-coordinates of the points to be the first and second components of the effective initial guesses (x_k0_list), respectively. Go.Layout is used decide the layout of the plot, including the title and axis titles. It also formats the y-axis to display only two decimal places.

In [ ]:
# Create traces
trace0 = go.Scatter(
    x = np.arange(len(x_k0_list)),
    y = x_k0_list[:, 0],
    mode = 'lines',
    name = 'x1_k0'
)

trace1 = go.Scatter(
    x = np.arange(len(x_k0_list)),
    y = x_k0_list[:, 1],
    mode = 'lines',
    name = 'x2_k0'
)

layout = go.Layout(
    title = 'Variation of Effective Initial Guesses',
    xaxis = dict(
        title = 'Iteration'
    ),
    yaxis = dict(
        title = 'Value',
        tickformat = '.2f'  # display only two decimal places
    )
)

data = [trace0, trace1]

# Create a figure
fig = go.Figure(data=data, layout=layout)

# Show the figure
fig.show()


The plot shows variation of effective initial guesses, x1_k0 and x2_k0, across the iterations of the optimization algorithm. The plot is showing values up to the 357th iteration because the algorithm then has found a solution satisfying the tolerance condition (f_star < tol) at that point and stopped. It means that the optimizer has succesfully found a solution that satisfies the 'tol' criteria. 

**Question 2:** Is it a better idea to set $\underline{K} = 100$? Is the convergence faster?

We uses the code from question 1. We changes the parameter $\underline{K} = 100$ and run the optimizer again. 
Again we plot the effective initial guesses $\mathbf{x}^{k0}$ vary with the iteration counter $k$.

In [ ]:
# Change of parameter
K_underline = 100

# Run the optimizer
x_star, x_k0_list = optimizer(griewank, bounds, tol, K_underline, K)

In [ ]:
# Create traces
trace0 = go.Scatter(
    x = np.arange(len(x_k0_list)),
    y = x_k0_list[:, 0],
    mode = 'lines',
    name = 'x1_k0'
)

trace1 = go.Scatter(
    x = np.arange(len(x_k0_list)),
    y = x_k0_list[:, 1],
    mode = 'lines',
    name = 'x2_k0'
)

layout = go.Layout(
    title = 'Variation of Effective Initial Guesses',
    xaxis = dict(
        title = 'Iteration'
    ),
    yaxis = dict(
        title = 'Value',
        tickformat = '.2f'  # display only two decimal places
    )
)

data = [trace0, trace1]

# Create a figure
fig = go.Figure(data=data, layout=layout)

# Show the figure
fig.show()

From the plot we see that the optimizer needs more iterations than in quesiton 1. For $\underline{K} = 100$ it needs 497 iterations compared to 357 in question 1. This means that the convergence is slower when only having $\underline{K} = 10$. 

As $\underline{K}$ refers to the number of "warm-up" iterations to find a good starting point for the main optimization process. Meaning that for $\underline{K} = 10$ the optimizer algorithm performs 10 warm-up iterations before the main optimization begins. So for a larger $\underline{K}$ the optimizer spends more iterations exploring the space randomly to  find a good starting point. 
If the function space is complex and has many local minima, it increases the chances that one of the warm-up iterations lands near the global minimum. 

This implies that having a larger $\underline{K}$ was more effective, indicating that this function space benefites from more warm-up iterations. 